**This notebook is to parse out pdb file and bin residues on the surface of the protein**

In [1]:
#import the necessary modules 
%matplotlib inline 
import numpy as np 
import matplotlib.pylab as plt 
import pandas as pd 
import scipy
import sklearn
import itertools 
from itertools import cycle 
import os.path as op
import timeit 


# These imports are for creating the percent occupancy heat map 
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

### create occupancy file using mdtraj and other tools from the .gro and .xtc file... Output txt file after splitting trajectory

### Figure out how to setup ipyparallel for using multiple nodes 

In [ ]:
import ipyparallel as ipp

## connect to the client
ipyclient = ipp.Client(profile="MPI32")

## print how many engines are connected
print(len(ipyclient), 'cores')


### Below makes a plot with number of surface residues 

In [ ]:
# Read in data from the oputput of wrapper.sh, where the frame count is given for each BSA residue that was within 
# 4 angstroms of PLGA trimer 
wat_data = pd.read_csv('occ_BSA1ns.txt', sep=" ", header=None, usecols=None ,index_col=None)
wat_data.columns = ["BSA_res_no","No. of frames"]
wat_data.head()

In [ ]:
a_a = ["GLY","ALA","VAL","LEU","ILE","MET","PHE","TRP","PRO","SER","THR","CYS","TYR","ASN","GLN","ASP"
               ,"GLU","LYS","ARG","HIS"]

In [ ]:
import multiprocessing as m_proc
m_proc.cpu_count()

In [ ]:
# Isolate each residue of bsa from pdb file(not in list method, 2nd draft, 70.72s run time) 

#start_time = timeit.default_timer()

red_bsa = []
for j in range(len(wat_data["BSA_res_no"])):
    for i in range(len(a_a)):
        st_r = str(" "+a_a[i]+"     "+str(j+1)+" ")
        st_r1 = str(" "+a_a[i]+"    "+str(j+1)+" ")
        st_r2 = str(" "+a_a[i]+"   "+str(j+1)+" ")
        for line in open("BSA.pdb",'r'):
            if st_r in line:
                # Since the list is empty, this if statement makes sure the first 10 residues are added properly 
                if len(red_bsa) == 0:
                    if st_r not in red_bsa:
                        l = line.splitlines()
                        red_bsa.append(l[0][16:27])
                elif len(red_bsa) != 0:
                    hj = len(red_bsa) - 1
                    if st_r not in red_bsa[hj]:
                        lb = line.splitlines()
                        red_bsa.append(lb[0][16:27])
            elif st_r1 in line:
                lj = len(red_bsa) - 1
                if st_r1 not in red_bsa[lj]:
                    b = line.splitlines()
                    red_bsa.append(b[0][16:27])     
            elif st_r2 in line:
                kj = len(red_bsa) - 1
                if st_r2 not in red_bsa[kj]:
                    r2 = line.splitlines()
                    red_bsa.append(r2[0][16:27]) 
                    
#timeit.default_timer() - start_time
#rb = [row[1:4] for row in red_bsa]

In [ ]:
red_bsa[0][4:9]

In [ ]:
# Isolate each residue of bsa from pdb file(memory intensive code, first draft, 73.52s to run through pdb file)

#s_time = timeit.default_timer()

red_bsa_mem = []
for j in range(len(wat_data["BSA_res_no"])):
    for i in range(len(a_a)):
        st_r = str(" "+a_a[i]+"     "+str(j+1)+" ")
        st_r1 = str(" "+a_a[i]+"    "+str(j+1)+" ")
        st_r2 = str(" "+a_a[i]+"   "+str(j+1)+" ")
        for line in open("../BSA.pdb",'r'):
            if st_r in line: 
                l = line.split()
                if len(l) == 10:
                    red_bsa_mem.append(l[3:5])
            elif st_r1 in line:
                b = line.split()
                if len(b) == 10:
                    red_bsa_mem.append(b[3:5])
            elif st_r2 in line:
                r2 = line.split()
                if len(r2) == 10:
                    red_bsa_mem.append(r2[3:5])
        res_l = len(red_bsa_mem)
        del red_bsa_mem[j+1:j+res_l]
        
#timeit.default_timer() - s_time

#rb = [row[0] for row in red_bsa]

In [ ]:
# This cell shows the frame count of each BSA residue 
rb = [row[1:4] for row in red_bsa] 
pand_rb = pd.Series(rb, index=None)
wat_data['BSA_res_des_order'] = pand_rb
wat_data = wat_data[["BSA_res_no",'BSA_res_des_order',"No. of frames"]]
wat_data

### Now onto to the counting of surface residues from occ.txt

In [ ]:
#Get the count of bsa residues that have 1001 or 1002 frames ( I ran a 1 ns NPT simulation of 1 BSA in water )

#aa_count = pd.DataFrame(data=a_a)
c_list = []

for i in range(len(a_a)):
    count = 0
    for index, row in wat_data.iterrows():
        if row['BSA_res_des_order'] == a_a[i]:
            if row['No. of frames'] == 1001: 
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames']))) 
            elif row['No. of frames'] == 1002:
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames'])))
    c_list.append(str(str(a_a[i])+"  "+str(count)))

In [ ]:
#aromatic_res = ['PHE', 'TRP', 'TYR', 'HIS']
hydrophobic_res = ['ALA', 'ILE', 'LEU', 'VAL', 'GLY', 'PRO','PHE', 'TRP','MET']
polar_res = ['ASN', 'CYS', 'GLN', 'SER', 'THR','TYR']
neg_res = ['ASP', 'GLU']
pos_res = ['ARG', 'HIS', 'LYS']

all_res = [pos_res, neg_res, polar_res, hydrophobic_res]
#Change order of residues before making the bar graph
# (!) Positively charged
# (2) Negatively charged
# (3) Polar residues 
# (4) Hydrophobic residues 


# This chunk of code sorts the counts of each AA that have 1001 or 1002 frame count based on the AA order in all_res
arr_list = []

for row in all_res:
    for i in range(len(c_list)):
        for j in range(len(row)):
            if row[j] == c_list[i][0:3]:
                arr_list.append(c_list[i])

#all_res
#arr_list

#This chunk of code splits the list arr_list to makes the AA: count of 1001 or 1002 frames data plottable 
f_list = []
fn_list = []
for i in range(len(arr_list)):
    f_list.append(arr_list[i][0:3])
    fn_list.append(int(arr_list[i][5:7]))

# Put the AA count in a pandas dataframe 
aa_count = pd.DataFrame(data=f_list, index=None, columns=['Amino_acids'])
new_lf = pd.Series(data=fn_list, index=None)
aa_count['No. of surf. res.'] = new_lf
#aa_count.tail()


In [ ]:
#plotting 
#Change order of residues before making the bar graph
# (!) Positively charged
# (2) Negatively charged
# (3) Polar residues 
# (4) Hydrophobic residues 
x = np.arange(20)
leg = ['Positively Charged', 'Negatively Charged', 'Polar', 'Hydrophobic']
plt.figure(figsize=(10,8))
plt.bar(x[0:3], aa_count['No. of surf. res.'][0:3], color='blue')
plt.bar(x[3:5], aa_count['No. of surf. res.'][3:5], color='red')
plt.bar(x[5:11], aa_count['No. of surf. res.'][5:11], color='magenta')
plt.bar(x[11:20], aa_count['No. of surf. res.'][11:20], color='cyan')
plt.xticks(x,aa_count['Amino_acids'])
plt.legend(leg)

In [ ]:
aa_count

In [ ]:
aa_count['No. of surf. res.'].sum()

### Now to calculate the fraction of contacts for each type amino acid group

In [ ]:
# Read in data from the oputput of wrapper.sh, where the frame count is given for each BSA residue that was within 
# 4 angstroms of PLGA trimer 
pl_25ns_data = pd.read_csv('occ_25ns_p1.txt', sep=" ", header=None, usecols=None ,index_col=None)
pl_25ns_data.columns = ['BSA_res_no','No_of_frames(PLGA trimer)']
#pand_rb = pd.Series(rb, index=None)
pl_25ns_data['BSA_res_des_order'] = pand_rb
pl_25ns_data = pl_25ns_data[["BSA_res_no","BSA_res_des_order","No_of_frames(PLGA trimer)"]]
pl_25ns_data.head()

## Use this occupancy file to organize surface amino acids similar to above code 

In [ ]:
pl_25ns_data[504:509]

In [ ]:
apl_list = []

# Some residues don't have any contact with PLGA trimers within 25 ns, put residues that do have contact with BSA in a separate list
for index, r_pl in pl_25ns_data.iterrows():
    if r_pl['No_of_frames(PLGA trimer)'] != 0:
        apl_list.append(r_pl['BSA_res_des_order'])

        
# This chunk of code gets an AA count from the above list, in order to get a total number of residues that contact BSA
cpl_l = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_list)):
        if apl_list[i] == r_a['Amino_acids']:
            count += 1
    cpl_l.append(count)      
        
aa_count['PLGA_trimer_25ns_SR'] = cpl_l
aa_count

In [ ]:
# This gives the total number of residues that are within 4 angstroms of PLGA trimers within a 25 ns block. 
# All other residues were not within 4 A over the course of the 25 ns trajectory block 
aa_count['PLGA_trimer_25ns_SR'].sum()

In [ ]:
# This gives the total number of residues that are within 4 angstroms of a water molecule within a 1 ns trajectory block
aa_count['No. of surf. res.'].sum()

In [ ]:
# Grouping of residues in Smith et al  
aromatic_res = ['PHE', 'TRP', 'TYR', 'HIS']
hydrophobic_res = ['ALA', 'ILE', 'LEU', 'VAL', 'GLY', 'PRO','PHE', 'TRP','MET']
polar_res = ['ASN', 'CYS', 'GLN', 'SER', 'THR','TYR']
neg_res = ['ASP', 'GLU']
pos_res = ['ARG', 'HIS', 'LYS']

frac_res = [aromatic_res, hydrophobic_res, neg_res, polar_res, pos_res]

# For each amino acid type in frac_res, this code chunk saves the frame count in a list and sums them together to 
# to get a total frame count within the 25 ns trajectory for each AA group in frac_res
co_grpaa = []

for row in frac_res:
    fr_list = []
    for j in range(len(row)):
        for index, r_pl in pl_25ns_data.iterrows():
            if row[j] == r_pl["BSA_res_des_order"]:
                fr_list.append(r_pl["No_of_frames(PLGA trimer)"])
    co_grpaa.append(sum(fr_list))

    
# For each AA type in frac_res, this code chunk saves the count for each AA within 4 Angstroms of a PLGA trimer 
# in a list based on the order in frac_res, then sums the counts to get a total number of AA for each AA type 
# that was within 4 Angstroms of a PLGA trimer 
tp_cnt = []
    
for row in frac_res:
    nw_l = []
    for j in range(len(row)):
        for index, r_aa in aa_count.iterrows(): 
            if row[j] == r_aa["Amino_acids"]:
                nw_l.append(r_aa['PLGA_trimer_25ns_SR'])
    tp_cnt.append(sum(nw_l))
    

# This data has not been normalized by each amino acid type 
res_aa = {'aa_group_type': ["Aromatic", "Hydrophobic", "Negative", "Polar", "Positive"]}
p25ns_NOT = pd.DataFrame(data=res_aa)
p25ns_NOT['frame_count'] = co_grpaa 
p25ns_NOT['type_of_contact'] = tp_cnt
p25ns_NOT

In [ ]:
# Now to calculate the normalized data 

# Total number of surface residues contacted by water in a only BSA/water simulation 
#b_sum = aa_count['No. of surf. res.'].sum()


# In Smith et al., the contacts data has been normalized by the protein
# surface fraction of each amino acid residue type contacted by each 
# polymer surrogate over the portion of the simulation trajectories 

# Get the total count of AA that are within 4 A of a PLGA trimer 
b_sum = p25ns_NOT['type_of_contact'].sum()


# The code chunk normalized the frame count of each AA group type by the protein surface fraction 
# of each amino acid type contacted by a polymer surrogate. It is possible that some amino acids 
# are in contact with more than 1 oligomer. This fractional contact does not account for that. 
norm_list = []

for index, row in p25ns_NOT.iterrows():
    norm_list.append(int(row["frame_count"]/(row["type_of_contact"]/b_sum)))
    
p25ns_NOT["contacts_NORM"] = norm_list 
p25ns_NOT.loc[5] = ["Sum", p25ns_NOT['frame_count'].sum(), p25ns_NOT['type_of_contact'].sum(), p25ns_NOT['contacts_NORM'].sum()]
p25ns_NOT

In [ ]:
# This conde chunk calculates the fractional contact based on the normalized frame count 
cont_l = []
f_sum = p25ns_NOT['contacts_NORM'].loc[5]

for index, row in p25ns_NOT.iterrows():
    cont_l.append(row['contacts_NORM']/f_sum)
    
p25ns_NOT['frac_cont_norm'] = cont_l
p25ns_NOT

In [ ]:
# This code plots the fractional contacts in a bar plot 
x = np.arange(5)
plt.figure(figsize=(10,10))
plt.bar(x, p25ns_NOT['frac_cont_norm'].loc[0:4])
plt.xticks(x,p25ns_NOT['aa_group_type'].loc[0:4])

### Now use MD Analysis to calculate no. of frames a PLGA residue was with 4 Angstroms of BSA

In [ ]:
# Import MDAnalysis
import MDAnalysis as mda
import MDAnalysis.analysis.distances as maa_dist

# Set up the MD simulation in a universe
u_plga = mda.Universe("nwsys_conf.gro","nowat_plgtri.xtc")

# Select protein
prot = u_plga.select_atoms("protein")

# Select PLG oligomers 
plga_tri = u_plga.select_atoms("resname PLG")


Visualize contact matrix with a 8 Angstroms cutoff

In [ ]:
p = u_plga.select_atoms("protein").positions
g = u_plga.select_atoms("resname PLG").positions

#Length units in MD Analysis is Angstroms, Using 8.5 A cutoff to better visualize atomic contacts
bs = maa_dist.contact_matrix(p,cutoff=8.0,box=u_plga.dimensions)
fig, ax = plt.subplots(figsize=(10,10))
l_s = ax.matshow(bs.astype(int), aspect="auto")
fig.colorbar(l_s, ax=ax)
ax.set_title('BSA Atomic Contact Map: PDB code 4F5S')

In [ ]:
# This code calculates how many atoms are in each residue of BSA and stores it in a dictionary 

# When saving new keys and their associated values in a dictionary, make sure the key is not the 
# same as a key that has already been saved. The value that has been saved with the key will be erased and the newest
# value will be placed in the dictionary if trying to add new value with the same key into the dict

rb_atom = {}
for i in range(len(red_bsa)):
    count = 0     
    st_r = red_bsa[i]
    for line in open("BSA.pdb",'r'):
        if st_r in line:
            count += 1
    rb_atom[st_r] = count

#rb_atom

In [ ]:
dist_pb[dist_pb <= np.float64(4)] = 1
dist_pb[dist_pb > np.float64(4)] = 0

In [ ]:
b = rb_atom[red_bsa[0]]
u_plga.atoms[0:b].center_of_mass()

### Distance-based analysis 

Find residues that have at least one atom within a cutoff $d = 4.0$ Angstrom near a PLGA trimer. 

In [ ]:
#Units of Angstroms 
dmax = 4.0 

In [ ]:
# Get all BSA residues, heavy atoms only
bsa_res = u_plga.select_atoms("protein and not type H")

# Get all PLGA trimers, heavy atoms only
plga_tri = u_plga.select_atoms("resname PLG and not name H")

In [ ]:
#Get distances 
dij_tri = maa_dist.distance_array(bsa_res.positions, plga_tri.positions, box=u_plga.trajectory.ts.dimensions)

In [ ]:
dij_tri[0]

In [ ]:
dij_tri.shape

In [ ]:
exp_bsa_atoms = np.any(dij_tri <= dmax, axis=1)

In [ ]:
# Boolean arrays must be of the same shape as the initial dimensions of the array being indexed.
# In the most straightforward case, the boolean array has the same shape
# Unlike in the case of integer index arrays, in the boolean case,
# the result is a 1-D array containing all the elements in the indexed array corresponding
#       to all the true elements in the boolean array.

bres_wit_dmxa = bsa_res[exp_bsa_atoms].residues

In [ ]:
list(bres_wit_dmxa)

In [ ]:
red_bsa[0][1:4]

In [ ]:
red_bsa[0][7:10]

In [ ]:
## Write a for loop that counts residues within bres_with_dmxa 

In [ ]:
## Define a function that does this, Courtesy of MDAnalysis tutorial 
def get_residues_dist(prot_atoms, g2_atoms, dmax):
    """Find all protein residues for which atoms that are within dmax"""
    dij_tri = maa_dist.distance_array(prot_atoms.positions, g2_atoms.positions, box=prot_atoms.universe.trajectory.ts.dimensions)
    exp_prot_atoms = np.any(dij_tri <= dmax, axis=1)
    return list(prot_atoms[exp_prot_atoms].residues)
    

In [ ]:
# This code uses center of mass distances to calculates whether PLGA trimers are within 4 A
alist = {}

for ts in u_plga.trajectory[0:50]:
    # Current atomic coordinates of BSA protein 
    br = u_plga.select_atoms("protein")
    
    # Current atomic coordinates of PLG residues 
    #a_plga = u_plga.select_atoms("resname PLG")
    
    # number of atoms in each PLG residue minus 1
    n = 48 
    
    # Number of trimers in simulation  
    n_oligs = 31
    
    #initialize numpy matrix to calculate COM PLGA coordinates 
    com_plg = np.zeros(shape=(31,3))
    
    #Initilaize numpy matrix to calculate of each BSA residue COM
    com_bres = np.zeros(shape=(583,3))
    
    #number of atoms in BSA protein 
    n_bsa_at = len(br)
    
    # There are 31 PLGA trimers in the simulation 
    for i in range(n_oligs):
        
        # Calculate the center of mass of each separate PLGA trimer
        com_plg[i] = u_plga.atoms[n_bsa_at:n_bsa_at+n].center_of_mass()
        
        # change the index to the next separate PLGA trimer 
        n_bsa_at = n_bsa_at + n 
        
    s_at = 0    

    # There are 583 residues in BSA    
    for i in range(len(red_bsa)):
        
        # From dictionary, save the no. of atoms for each residue in a variable
        at_no = rb_atom[red_bsa[i]]
        
        # Calculate the center of mass of each BSa residue 
        com_bres[i] = u_plga.atoms[s_at:s_at+at_no].center_of_mass()
        
        # Change the index to the next BSA residue 
        s_at = s_at + at_no
    
    #Calculation of distance matrix for atom contact pairs under the 4 A cutoff 
    prs, dmat = mda.lib.distances.capped_distance(com_bres,com_plg,max_cutoff=4.0, box=u_plga.dimensions)

    # Calculation of the distance matrix between COM of plga trimers and COM of each BSA residue 
    dd_rc = mda.lib.distances.distance_array(com_bres, com_plg, box=u_plga.dimensions)

    # Filter out distances under 4 A 
    dd_rc[dd_rc <= np.float64(5)] = 1
    dd_rc[dd_rc > np.float64(5)] = 0 
    
    print(prs)
    
    # For the distances, find the BSA residue number and change counter 
    # This code chunk works properply
    for row in prs:
        count = 0
        for i in range(len(red_bsa)): 
            #print(i)
            nk = row[0] + 1
            if nk == np.int64(red_bsa[i][7:10]):
                if bool(alist) == False:
                    count += 1
                    alist[red_bsa[i]] = count
                    print(alist)
                elif bool(alist) != False:
                    count += 1
                    b = list(alist.keys())
                    if red_bsa[i] in b: 
                        nw = int(alist[red_bsa[i]]) + count
                        alist[red_bsa[i]] = nw
                        print(alist)
                    elif red_bsa[i] not in b:
                        alist[red_bsa[i]] = count  
                        print(alist)

In [ ]:
alist

In [ ]:
# This contact map is using previous simulation of 31 PLGA trimers and BSA in water. Contact matrix using COM distances, 
# VMD uses any atom within 4 is the criteria 
fig, ax = plt.subplots(figsize=(10,10))
com_ls = ax.matshow(dd_rc, aspect="auto")
fig.colorbar(com_ls, ax=ax)
ax.set_title('BSA plga trimer COM contact map')

#### Calculate the autocorrelation of a variable in time 